# Data Handling

In [ ]:
# Function to display array shapes in a table
def display_array_shapes(images, labels):
    # Get the shapes of the arrays
    image_shape = images.shape
    label_shape = labels.shape

    # Create a list of data to display in the table
    data = [("Images", image_shape), ("Labels", label_shape)]

    # Print the data in a tabular format
    table = tabulate(data, headers=["Array Name", "Shape"], tablefmt="grid")
    print(table)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# Function to binarize labels
def binarize_labels(labels):
    label_binarizer = LabelBinarizer()
    labels_binarized = label_binarizer.fit_transform(labels)
    return labels_binarized

In [ ]:
from collections import Counter

# Function to print label counts
def display_label_counts(label_counts):
    print("Label\tCount")
    print("----------------")
    for label, count in label_counts.items():
        print(f"{label}\t{count}")

In [ ]:
from skimage.transform import resize

# Function to resize images
def resize_images(images, image_size):
    # Check if images is a numpy array
    if not isinstance(images, np.ndarray):
        images = np.array(images)

    # Initialize list to store preprocessed images
    preprocessed_images = []

    for img in images:
        # Resize image
        img = resize(img, image_size)

        preprocessed_images.append(img)

    # Return preprocessed images as a numpy array
    return np.array(preprocessed_images)

# Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def augment_data(images, labels, label_counts, augmentation_factor=3):
    # Find labels with less than 75 images
    labels_to_augment = [label for label, count in label_counts.items() if count < 75]

    # Create an ImageDataGenerator for augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Augment images
    augmented_images = []
    augmented_labels = []

    for label in labels_to_augment:
        label_indices = np.where(labels == label)[0]
        images_to_augment = images[label_indices]
        for _ in range(augmentation_factor):
            for image in images_to_augment:
                augmented_image = datagen.random_transform(image)
                augmented_images.append(augmented_image)
                augmented_labels.append(label)

    # Convert the lists to NumPy arrays
    augmented_images = np.array(augmented_images)
    augmented_labels = np.array(augmented_labels)

    # Display the counts of the augmented labels
    print("Label\tCount")
    print("----------------")
    for label in labels_to_augment:
        count = np.sum(augmented_labels == label)
        print(f"{label}\t{count}")

    return augmented_images, augmented_labels

In [ ]:
def combine_augmented_data(images, labels, augmented_images, augmented_labels):
    # Find labels with less than 75 images
    labels_to_augment = [label for label, count in label_counts.items() if count < 75]

    # Find indices of labels to augment
    indices_to_augment = np.where(np.isin(labels, labels_to_augment))[0]

    # Remove original images and labels that are being augmented
    images_remaining = np.delete(images, indices_to_augment, axis=0)
    labels_remaining = np.delete(labels, indices_to_augment, axis=0)

    # Combine the augmented images and labels with the remaining images and labels
    augmented_images_combined = np.concatenate((images_remaining, augmented_images))
    augmented_labels_combined = np.concatenate((labels_remaining, augmented_labels))

    # Shuffle the combined data
    shuffle_indices = np.random.permutation(len(augmented_labels_combined))
    augmented_images_combined_shuffled = augmented_images_combined[shuffle_indices]
    augmented_labels_combined_shuffled = augmented_labels_combined[shuffle_indices]

    # Verify the shapes of the combined arrays
    print("Original images shape:", images.shape)
    print("Original labels shape:", labels.shape)
    print("Augmented images shape:", augmented_images.shape)
    print("Augmented labels shape:", augmented_labels.shape)
    print("Remaining images shape:", images_remaining.shape)
    print("Remaining labels shape:", labels_remaining.shape)
    print("Augmented images combined shape:", augmented_images_combined.shape)
    print("Augmented labels combined shape:", augmented_labels_combined.shape)

    return augmented_images_combined_shuffled, augmented_labels_combined_shuffled

# Graph Plotting

In [ ]:
import matplotlib.pyplot as plt

# Function to plot training history
def plot_training_history(history):
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Time')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Over Time')
    plt.legend()

    plt.show()

# Building CNN Models

In [ ]:
# Function to build and compile the VGG19 model
def build_and_compile_vgg19_model(num_classes):
    # Load the VGG19 model with pre-trained weights and exclude the top classification layer
    vgg19_base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze the pre-trained layers
    for layer in vgg19_base_model.layers:
        layer.trainable = False

    # Create the final classification layers
    x = tf.keras.layers.Flatten()(vgg19_base_model.output)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Create the final model
    vgg19_model = tf.keras.Model(inputs=vgg19_base_model.input, outputs=predictions)

    # Compile the model
    vgg19_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return vgg19_model

In [ ]:
# Function to build and compile the ResNet101 model
def build_and_compile_resnet101_model(num_classes):
    # Load the ResNet101 model with pre-trained weights and exclude the top classification layer
    resnet101_base_model = tf.keras.applications.ResNet101(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze the pre-trained layers
    for layer in resnet101_base_model.layers:
        layer.trainable = False

    # Create the final classification layers
    x = tf.keras.layers.Flatten()(resnet101_base_model.output)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Create the final model
    resnet101_model = tf.keras.Model(inputs=resnet101_base_model.input, outputs=predictions)

    # Compile the model
    resnet101_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return resnet101_model

In [ ]:
# Function to build and compile the InceptionV3 model
def build_and_compile_inceptionv3_model(num_classes):
    # Load the InceptionV3 model with pre-trained weights and exclude the top classification layer
    inceptionv3_base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

    # Freeze the pre-trained layers
    for inceptionv3_layer in inceptionv3_base_model.layers:
        inceptionv3_layer.trainable = False

    # Create the final classification layers
    x = tf.keras.layers.Flatten()(inceptionv3_base_model.output)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Create the final model
    inceptionv3_model = tf.keras.Model(inputs=inceptionv3_base_model.input, outputs=predictions)

    # Compile the model
    inceptionv3_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return inceptionv3_model

In [ ]:
# Function to build and compile the MobileNetV2 model
def build_and_compile_mobilenetv2_model(num_classes):
    # Load the MobileNetV2 model with pre-trained weights and exclude the top classification layer
    mobilenetv2_base_model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze the pre-trained layers
    for mobilenetv2_layer in mobilenetv2_base_model.layers:
        mobilenetv2_layer.trainable = False

    # Create the final classification layers
    x = tf.keras.layers.Flatten()(mobilenetv2_base_model.output)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Create the final model
    mobilenetv2_model = tf.keras.Model(inputs=mobilenetv2_base_model.input, outputs=predictions)

    # Compile the model
    mobilenetv2_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return mobilenetv2_model

In [ ]:
# Function to build and compile the EfficientNet model
def build_and_compile_efficientnet_model(num_classes):
    # Load the EfficientNetB0 model with pre-trained weights and exclude the top classification layer
    efficientnet_base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    # Freeze the pre-trained layers
    for efficientnet_layer in efficientnet_base_model.layers:
        efficientnet_layer.trainable = False

    # Create the final classification layers
    x = tf.keras.layers.Flatten()(efficientnet_base_model.output)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # Create the final model
    efficientnet_model = tf.keras.Model(inputs=efficientnet_base_model.input, outputs=predictions)

    # Compile the model
    efficientnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return efficientnet_model

# Evaluating Model

In [ ]:
# Function to get predictied and true labels lists
def make_predictions_and_evaluate(model, X_data, y_true):
    # Set the model to evaluation mode
    model.evaluate(X_data, y_true)

    # Make predictions on the dataset
    predicted_probabilities = model.predict(X_data)

    # Convert predicted probabilities to class labels
    predicted_labels = np.argmax(predicted_probabilities, axis=1)

    # Convert true labels to a list
    true_labels_list = np.argmax(y_true, axis=1)

    return predicted_labels, true_labels_list

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Function to calculate classification metrics
def calculate_classification_metrics(true_labels_list, predicted_labels):
    # Calculate precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels_list,
        predicted_labels,
        average=None,
        zero_division=0
    )

    # Calculate average precision, recall, and F1 score
    avg_precision = precision.mean()
    avg_recall = recall.mean()
    avg_f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0

    # Calculate accuracy
    accuracy = accuracy_score(true_labels_list, predicted_labels)

    # Print the average metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall: {avg_recall:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")

In [ ]:
# Function to display confusion matrix
def display_confusion_matrix(true_labels_list, predicted_labels):
    confusion = confusion_matrix(true_labels_list, predicted_labels)

    # Display confusion matrix as a heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)

    # Set ticks at positions 1, 2, ..., n
    classes = range(1, len(confusion) + 1)
    plt.xticks(classes, labels=classes)
    plt.yticks(classes, labels=classes)

    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()